In [1]:
import pandas as pd
import yfinance as yf

In [2]:
news = pd.read_csv('apple_finbert.csv')

In [3]:
news = news.drop(['Unnamed: 0'], axis=1)

In [4]:
news['DATE'] = pd.to_datetime(news['DATE'])

In [5]:
news['DATE'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 2177 entries, 0 to 2176
Series name: DATE
Non-Null Count  Dtype         
--------------  -----         
2177 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 17.1 KB


In [6]:
news = news.groupby('DATE')['IMPACT_SCORE'].sum().reset_index()

In [7]:
max_score = news['IMPACT_SCORE'].max()
min_score = news['IMPACT_SCORE'].min()
max_score, min_score

(6.4679176937, -5.995869565)

In [8]:
news

,DATE,IMPACT_SCORE
0,2021-01-01,1.011975
1,2021-01-02,1.060055
2,2021-01-03,0.238335
3,2021-01-04,1.585697
4,2021-01-05,1.722996
...,...,...
362,2022-01-04,1.643082
363,2022-01-05,0.059643
364,2022-01-06,0.006974
365,2022-01-07,1.806651


In [9]:
stock = yf.download('AAPL')

[*********************100%%**********************]  1 of 1 completed


In [10]:
stock = stock.loc['2020-12-29':'2022-01-10']
stock = stock.reset_index()


In [11]:
stock['Date'] = pd.to_datetime(stock['Date'])
stock['Date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 261 entries, 0 to 260
Series name: Date
Non-Null Count  Dtype         
--------------  -----         
261 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.2 KB


In [12]:
stock = stock.rename(columns={'Date': 'DATE'})
stock

,DATE,Open,High,Low,Close,Adj Close,Volume
0,2020-12-29,138.050003,138.789993,134.339996,134.869995,132.028397,121047300
1,2020-12-30,135.580002,135.990005,133.399994,133.720001,130.902634,96452100
2,2020-12-31,134.080002,134.740005,131.720001,132.690002,129.894318,99116600
3,2021-01-04,133.520004,133.610001,126.760002,129.410004,126.683411,143301900
4,2021-01-05,128.889999,131.740005,128.429993,131.009995,128.249710,97664900
...,...,...,...,...,...,...,...
256,2022-01-04,182.630005,182.940002,179.119995,179.699997,176.998337,99310400
257,2022-01-05,179.610001,180.169998,174.639999,174.919998,172.290222,94537600
258,2022-01-06,172.699997,175.300003,171.639999,172.000000,169.414124,96904000
259,2022-01-07,172.889999,174.139999,171.029999,172.169998,169.581558,86709100


In [13]:
combined_df = pd.merge(news, stock, on='DATE', how='inner')
combined_df

,DATE,IMPACT_SCORE,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,1.585697,133.520004,133.610001,126.760002,129.410004,126.683411,143301900
1,2021-01-05,1.722996,128.889999,131.740005,128.429993,131.009995,128.249710,97664900
2,2021-01-06,1.212556,127.720001,131.050003,126.379997,126.599998,123.932640,155088000
3,2021-01-07,1.788645,128.360001,131.630005,127.860001,130.919998,128.161636,109578200
4,2021-01-08,-1.441645,132.429993,132.630005,130.229996,132.050003,129.267792,105158200
...,...,...,...,...,...,...,...,...
251,2022-01-03,6.467918,177.830002,182.880005,177.710007,182.009995,179.273605,104487900
252,2022-01-04,1.643082,182.630005,182.940002,179.119995,179.699997,176.998337,99310400
253,2022-01-05,0.059643,179.610001,180.169998,174.639999,174.919998,172.290222,94537600
254,2022-01-06,0.006974,172.699997,175.300003,171.639999,172.000000,169.414124,96904000


In [14]:
combined_df['stock_pattern'] = (combined_df['Close'].shift(-1) >= combined_df['Open']).astype(int)

In [15]:
value_counts = combined_df['stock_pattern'].value_counts()
value_counts

stock_pattern
1    138
0    118
Name: count, dtype: int64

In [16]:
combined_df = combined_df.drop(['Open', 'Low', 'High', 'Close', 'Adj Close', 'Volume'], axis=1)

In [17]:
combined_df.to_csv('Train_Test.csv', index=False)

In [18]:
data = pd.read_csv('Train_Test_AAPL.csv')

In [19]:
data

,DATE,IMPACT_SCORE,stock_pattern
0,2021-01-04,1.585697,0
1,2021-01-05,1.722996,0
2,2021-01-06,1.212556,1
3,2021-01-07,1.788645,1
4,2021-01-08,-1.441645,0
...,...,...,...
251,2022-01-03,6.467918,1
252,2022-01-04,1.643082,0
253,2022-01-05,0.059643,0
254,2022-01-06,0.006974,0
